In [1]:
import pandas as pd
import json

# --- 1. Simulação de Carregamento de Dados (Convertendo para Pandas DataFrames) ---

# Dados de candidatos
candidatos_data = {
    "123": {
        "infos_basicas": {"nome": "João Silva"},
        "informacoes_profissionais": {
            "titulo_profissional": "Desenvolvedor Backend Sênior",
            "area_atuacao": "Tecnologia",
            "conhecimentos_tecnicos": "Python, Django, Flask, SQL, AWS, Docker, Kubernetes",
            "nivel_profissional": "Sênior",
        },
        "formacao_e_idiomas": {
            "nivel_academico": "Ensino Superior Completo",
            "nivel_ingles": "Avançado"
        },
        "cv_pt": "Experiência de 5 anos como Desenvolvedor Backend Sênior, com foco em sistemas distribuídos e microserviços. Atuou em projetos utilizando Python, Django e Flask para APIs RESTful. Conhecimento sólido em bancos de dados SQL (PostgreSQL, MySQL) e NoSQL (MongoDB). Experiência com implantação e monitoramento em AWS, utilizando EC2, S3, RDS. Familiaridade com Docker e Kubernetes para orquestração de contêineres. Participou do ciclo completo de desenvolvimento de software, incluindo design, implementação, testes e deploy. Certificado em AWS Solutions Architect Associate."
    },
    "456": {
        "infos_basicas": {"nome": "Maria Souza"},
        "informacoes_profissionais": {
            "titulo_profissional": "Analista de Dados Júnior",
            "area_atuacao": "Dados e Analytics",
            "conhecimentos_tecnicos": "SQL, Python, Pandas, Tableau, Power BI",
            "nivel_profissional": "Júnior",
        },
        "formacao_e_idiomas": {
            "nivel_academico": "Ensino Superior Incompleto",
            "nivel_ingles": "Intermediário"
        },
        "cv_pt": "Estudante de Ciência da Computação, com 2 anos de experiência como Analista de Dados em estágio e projetos acadêmicos. Proficiência em SQL para manipulação de grandes datasets e experiência com Python para análise de dados utilizando Pandas e NumPy. Criação de dashboards interativos em Tableau e Power BI. Interesse em machine learning e otimização de processos de dados."
    }
}

# Achatar e converter candidatos_data para DataFrame
df_candidatos = pd.DataFrame.from_dict(candidatos_data, orient='index')
# Expandir as colunas aninhadas (infos_basicas, informacoes_profissionais, formacao_e_idiomas)
df_candidatos = pd.json_normalize(df_candidatos.to_dict(orient='records'))
# Renomear a coluna 'codigo_profissional' se ela existir, ou criar uma 'codigo' com o índice.
# No seu exemplo, o ID do candidato era a chave do dicionário, vamos mantê-lo como uma coluna 'codigo'.
df_candidatos['codigo'] = candidatos_data.keys()
df_candidatos.set_index('codigo', inplace=True)

# print("DataFrame de Candidatos:")
# print(df_candidatos.head())
# print("\n")

# Dados de vagas
vagas_data = {
    "VAGA001": {
        "informacoes_basicas": {"titulo_vaga": "Desenvolvedor Backend Sênior Python"},
        "perfil_vaga": {
            "nivel_profissional": "Sênior",
            "areas_atuacao": "Desenvolvimento de Software",
            "principais_atividades": "Desenvolver e manter APIs RESTful escaláveis em Python (Django/Flask). Projetar e implementar soluções de banco de dados. Integrar com serviços de cloud (AWS). Participar de code reviews e disseminar boas práticas de desenvolvimento.",
            "competencia_tecnicas_e_comportamentais": "Sólidos conhecimentos em Python, Django/Flask. Experiência com Docker e Kubernetes. Conhecimento em SQL. Boas habilidades de comunicação e trabalho em equipe. Proatividade na resolução de problemas.",
            "nivel_ingles": "Avançado"
        }
    },
    "VAGA002": {
        "informacoes_basicas": {"titulo_vaga": "Analista de Dados Júnior"},
        "perfil_vaga": {
            "nivel_profissional": "Júnior",
            "areas_atuacao": "Análise de Dados",
            "principais_atividades": "Coletar, limpar e analisar dados de diversas fontes. Criar relatórios e dashboards para stakeholders. Identificar tendências e insights a partir dos dados. Desenvolver scripts em SQL e Python para automação de tarefas.",
            "competencia_tecnicas_e_comportamentais": "Conhecimento em SQL e Python (Pandas). Habilidade com ferramentas de BI (Tableau/Power BI). Capacidade analítica e atenção aos detalhes. Boa comunicação para apresentar resultados.",
            "nivel_ingles": "Intermediário"
        }
    }
}

# Achatar e converter vagas_data para DataFrame
df_vagas = pd.DataFrame.from_dict(vagas_data, orient='index')
df_vagas = pd.json_normalize(df_vagas.to_dict(orient='records'))
df_vagas['codigo_vaga'] = vagas_data.keys()
df_vagas.set_index('codigo_vaga', inplace=True)

# print("DataFrame de Vagas:")
# print(df_vagas.head())
# print("\n")

# Dados de prospects/candidaturas
prospects_data = {
    "PROSP001": {
        "titulo": "Desenvolvedor Backend Sênior Python",
        "prospects": [
            {
                "nome": "João Silva",
                "codigo": "123",
                "situacao_candidado": "Encaminhado ao Requisitante",
                "comentario": "Candidato com perfil aderente, experiência robusta em Python e AWS. Possui certificação AWS."
            }
        ]
    },
    "PROSP002": {
        "titulo": "Analista de Dados Júnior",
        "prospects": [
            {
                "nome": "Maria Souza",
                "codigo": "456",
                "situacao_candidado": "Prospect",
                "comentario": "Estudante promissora com bons fundamentos em SQL e Python. Apresentou portfólio interessante."
            }
        ]
    }
}

# Achatar e converter prospects_data para DataFrame
# Isso requer um loop para lidar com a lista aninhada de "prospects"
all_prospects = []
for prosp_id, data in prospects_data.items():
    titulo_vaga = data['titulo']
    for prospect_detail in data['prospects']:
        flat_prospect = {
            'prospect_id': prosp_id,
            'titulo_vaga_prospect': titulo_vaga, # Nome diferente para evitar conflito com 'titulo_vaga' de vagas_data
            **prospect_detail
        }
        all_prospects.append(flat_prospect)

df_prospects = pd.DataFrame(all_prospects)
# Definindo 'codigo' do candidato como índice para facilitar buscas
df_prospects.set_index('codigo', inplace=True)

# print("DataFrame de Prospects:")
# print(df_prospects.head())
# print("\n")


# --- 2. Função RAG para Geração de Perguntas (Adaptada para Pandas) ---

def gerar_pergunta_rag_pandas(candidato_id: str, df_candidatos: pd.DataFrame, df_vagas: pd.DataFrame, df_prospects: pd.DataFrame):
    """
    Simula o processo RAG para gerar uma pergunta de entrevista usando DataFrames Pandas.
    """
    # Recuperar informações do candidato
    candidato = df_candidatos.loc[candidato_id]

    # Recuperar informações de prospects relacionadas ao candidato
    prospect_info = df_prospects.loc[candidato_id]
    
    # Usar o titulo_vaga do prospect para buscar a vaga correspondente
    vaga_titulo = prospect_info['titulo_vaga_prospect']
    vaga = df_vagas[df_vagas['informacoes_basicas.titulo_vaga'] == vaga_titulo].iloc[0]

    if candidato.empty or vaga.empty:
        return "Erro: Candidato ou vaga não encontrados."

    # Informações para contextualizar o LLM (contexto de recuperação)
    contexto_candidato = f"""
    Nome: {candidato['infos_basicas.nome']}
    Título Profissional: {candidato['informacoes_profissionais.titulo_profissional']}
    Área de Atuação: {candidato['informacoes_profissionais.area_atuacao']}
    Conhecimentos Técnicos: {candidato['informacoes_profissionais.conhecimentos_tecnicos']}
    Nível Profissional: {candidato['informacoes_profissionais.nivel_profissional']}
    Inglês: {candidato['formacao_e_idiomas.nivel_ingles']}
    CV Resumo: {candidato['cv_pt'][:500]}... # Primeiros 500 caracteres do CV
    """

    contexto_vaga = f"""
    Título da Vaga: {vaga['informacoes_basicas.titulo_vaga']}
    Nível Profissional Requerido: {vaga['perfil_vaga.nivel_profissional']}
    Principais Atividades: {vaga['perfil_vaga.principais_atividades']}
    Competências Técnicas e Comportamentais: {vaga['perfil_vaga.competencia_tecnicas_e_comportamentais']}
    Inglês Requerido: {vaga['perfil_vaga.nivel_ingles']}
    Observações Adicionais: {vaga['perfil_vaga.demais_observacoes'] if 'perfil_vaga.demais_observacoes' in vaga and pd.notna(vaga['perfil_vaga.demais_observacoes']) else 'N/A'}
    """

    contexto_prospect = f"""
    Situação da Candidatura: {prospect_info['situacao_candidado']}
    Comentário do Recrutador Anterior: {prospect_info['comentario'] if pd.notna(prospect_info['comentario']) and prospect_info['comentario'] else 'Nenhum comentário.'}
    """

    # Prompt para o LLM (simulado)
    prompt_llm = f"""
    Você é um(a) entrevistador(a) de IA.
    Com base nas seguintes informações do candidato, da vaga e do histórico de prospecção,
    formule uma pergunta de entrevista relevante e perspicaz para o candidato.
    Foque em explorar a experiência prática e as competências necessárias para a vaga.

    --- Informações do Candidato ---
    {contexto_candidato}

    --- Informações da Vaga ---
    {contexto_vaga}

    --- Histórico da Prospecção ---
    {contexto_prospect}

    --- Pergunta a Ser Gerada ---
    """

    # Simulação da resposta do LLM (o 'Generator' do RAG)
    if "Desenvolvedor Backend" in vaga_titulo and "Python" in candidato['informacoes_profissionais.conhecimentos_tecnicos']:
        pergunta = f"Olá {candidato['infos_basicas.nome']}, seu CV menciona experiência com Python e Django/Flask. Poderia descrever um projeto desafiador onde você utilizou essas tecnologias para desenvolver APIs RESTful escaláveis e como você lidou com os desafios de performance e integração em ambientes de nuvem como AWS?"
    elif "Analista de Dados" in vaga_titulo and "SQL" in candidato['informacoes_profissionais.conhecimentos_tecnicos']:
        pergunta = f"Olá {candidato['infos_basicas.nome']}, você menciona experiência com SQL e ferramentas de BI. Poderia compartilhar um exemplo de como você transformou dados brutos em insights acionáveis para um projeto ou negócio, e qual foi a ferramenta de BI que mais contribuiu para essa apresentação?"
    else:
        pergunta = f"Olá {candidato['infos_basicas.nome']}, com base em seu perfil e na vaga de {vaga_titulo}, poderia nos contar mais sobre sua experiência mais relevante para este cargo?"

    return pergunta

# --- 3. Exemplo de Uso com DataFrames Pandas ---

print("--- Exemplo de Pergunta para João Silva (Desenvolvedor Backend Sênior) - Usando Pandas ---")
# Para PROSP001, o código do candidato é "123"
pergunta_joao_pandas = gerar_pergunta_rag_pandas("123", df_candidatos, df_vagas, df_prospects)
print(pergunta_joao_pandas)
print("\n" + "="*80 + "\n")

print("--- Exemplo de Pergunta para Maria Souza (Analista de Dados Júnior) - Usando Pandas ---")
# Para PROSP002, o código do candidato é "456"
pergunta_maria_pandas = gerar_pergunta_rag_pandas("456", df_candidatos, df_vagas, df_prospects)
print(pergunta_maria_pandas)
print("\n" + "="*80 + "\n")

--- Exemplo de Pergunta para João Silva (Desenvolvedor Backend Sênior) - Usando Pandas ---
Olá João Silva, seu CV menciona experiência com Python e Django/Flask. Poderia descrever um projeto desafiador onde você utilizou essas tecnologias para desenvolver APIs RESTful escaláveis e como você lidou com os desafios de performance e integração em ambientes de nuvem como AWS?


--- Exemplo de Pergunta para Maria Souza (Analista de Dados Júnior) - Usando Pandas ---
Olá Maria Souza, você menciona experiência com SQL e ferramentas de BI. Poderia compartilhar um exemplo de como você transformou dados brutos em insights acionáveis para um projeto ou negócio, e qual foi a ferramenta de BI que mais contribuiu para essa apresentação?




In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA device count: {torch.cuda.device_count()}")
else:
    print("CUDA is not available. PyTorch will run on CPU.")

# Exemplo simples para verificar se um tensor pode ser movido para a GPU
if torch.cuda.is_available():
    x = torch.rand(3, 3).cuda() # Cria um tensor na GPU
    print(f"Tensor on GPU: {x}")
    print(f"Device of tensor: {x.device}")
else:
    print("Cannot create a tensor on GPU as CUDA is not available.")

PyTorch version: 2.7.0+cpu
CUDA available: False
CUDA is not available. PyTorch will run on CPU.
Cannot create a tensor on GPU as CUDA is not available.


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA device count: {torch.cuda.device_count()}")
else:
    print("CUDA is not available. PyTorch will run on CPU.")

# Exemplo simples para verificar se um tensor pode ser movido para a GPU
if torch.cuda.is_available():
    x = torch.rand(3, 3).cuda() # Cria um tensor na GPU
    print(f"Tensor on GPU: {x}")
    print(f"Device of tensor: {x.device}")
else:
    print("Cannot create a tensor on GPU as CUDA is not available.")

PyTorch version: 2.7.0+cpu
CUDA available: False
CUDA is not available. PyTorch will run on CPU.
Cannot create a tensor on GPU as CUDA is not available.


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")

    try:
        x = torch.tensor([1, 2, 3], device='cuda:0')
        print(f"Tensor on GPU: {x}")
        print(f"Device of tensor: {x.device}")
    except RuntimeError as e:
        print(f"Erro ao tentar criar tensor na GPU: {e}")
        print("Isso pode indicar que a GPU não está acessível pelo PyTorch,")
        print("mesmo que 'torch.cuda.is_available()' seja True.")
else:
    print("CUDA is NOT available. PyTorch will run on CPU.")
    print("Verifique a instalação dos drivers NVIDIA, CUDA Toolkit e cuDNN.")

print("\n--- Verificação de uma operação simples na GPU (se disponível) ---")
if torch.cuda.is_available():
    try:
        a = torch.randn(2, 2).to('cuda')
        b = torch.randn(2, 2).to('cuda')
        c = a @ b
        print(f"Resultado de operação na GPU:\n{c}")
    except Exception as e:
        print(f"Erro durante operação na GPU: {e}")
else:
    print("GPU não disponível para testes de operação.")

PyTorch version: 2.7.0+cpu
CUDA available: False
CUDA is NOT available. PyTorch will run on CPU.
Verifique a instalação dos drivers NVIDIA, CUDA Toolkit e cuDNN.

--- Verificação de uma operação simples na GPU (se disponível) ---
GPU não disponível para testes de operação.
